#Создание собственной библиотеки автоматического дифференцирования

In [1]:
import math
import numpy as np

In [283]:
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None # function
        self._prev = set(_children) # set of Value objects
        self._op = _op # the op that produced this node, string ('+', '-', ....)

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(data = self.data+other.data,
                    _children = (self, other),
                    _op = "+")
        def _backward():
            if out.grad == None:
                self.grad = other.grad*out.grad
                other.grad = self.grad*out.grad
            else:
                self.grad+=out.grad
                other.grad+=out.grad
            return self.grad+other.grad
        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other
    def __neg__(self): # -self
        return self * -1
    def __sub__(self, other): # self - other
        return self + (-other)
    def __rsub__(self, other): # other - self
        return other + (-self)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(data = self.data*other.data,
                    _children = (self, other),
                    _op = "*")
        def _backward():
            if out.grad == None:
                self.grad = other.data*out.grad
                other.grad = self.data*out.grad
            else:
                self.grad+=out.grad*other.data
                other.grad+=out.grad*self.data
        out._backward = _backward
        return out

    def __rmul__(self, other): # other * self
        return self * other
    def __truediv__(self, other): # self / other
        return self * other**-1
    def __rtruediv__(self, other): # other / self
        return other * self**-1
    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __pow__(self, other):
        assert isinstance(other, (float, int)), "only supporting int/floats power now"
        out = Value(data = self.data**other,
                    _children = (self, self),
                    _op = "**")
        def _backward():
            if out.grad == None:
                self.grad = (self.data ** (other-1))*(other) * out.grad
            else:
              self.grad+= (self.data ** (other-1))*(other) * out.grad
        out._backward = _backward
        return out

    def __relu__(self):
        out = Value(max(self.data, 0),
                    _children = (self, self),
                    _op = "&")
        def _backward():
            if out.grad == None:
                self.grad = min(self.data, 0)
            else:
                self.grad+=min(self.data, 0) * out.grad
        out._backward = _backward
        return out


    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()



In [284]:
import torch
def my_tests():
    x = Value(-4.0)
    temp_z1 = 2. + x
    temp_z2 = x + 2.
    temp_z3 = -2. + x
    temp_z4 = 2. * x
    temp_z5 = x* 2.
    z = 2 * x + 2 + x
    z.backward()
    temp_xmg, temp_zmg = x, z

    x = torch.Tensor([-4.0]).double()
    x.requires_grad = True
    z = 2 * x + 2 + x
    z.backward()
    temp_xpt, temp_zpt = x, z

    # forward pass went well
    assert temp_zmg.data == temp_zpt.data.item()
    # backward pass went well
    print(temp_xmg, temp_xpt, temp_xpt.grad)
    assert temp_xmg.grad == temp_xpt.grad.item()

    y = Value(-4.0)
    temp_h1 = y**4 + 2
    temp_h2 = temp_h1
    temp_h2.backward()
    temp_ypowm, temp_zpowm = y, temp_h2

    y = torch.Tensor([-4.0]).double()
    y.requires_grad = True
    h2 = y**4 + 2
    h2.backward()
    temp_ypowpt, temp_zpowpt = y, h2
    print(temp_zpowm.data)
    print(temp_zpowpt.data.item())
    # forward pass went well
    assert temp_zpowm.data == temp_zpowpt.data.item()
    # backward pass went well
    print(temp_ypowm)
    print(temp_ypowpt, temp_ypowpt.grad)
    assert temp_ypowm.grad == temp_ypowpt.grad.item()

In [285]:
my_tests()

Value(data=-4.0, grad=3) tensor([-4.], dtype=torch.float64, requires_grad=True) tensor([3.], dtype=torch.float64)
258.0
258.0
Value(data=-4.0, grad=-256.0)
tensor([-4.], dtype=torch.float64, requires_grad=True) tensor([-256.], dtype=torch.float64)


In [295]:
def test_sanity_check():

    x = Value(-4.0)
    z_5 = 2.+x
    z_2 = x+2.
    z_3 = x*2.
    z_4 = 2.*x
    z = 2 * x + 2 + x

    q = z + z * x                       # Не понял как правильно задать параметр _op для .relu()
    h = (z * z)
    y = h + q + q * x
    y.backward()
    xmg, ymg = x, y

    x = torch.Tensor([-4.0]).double()
    x.requires_grad = True
    z = 2 * x + 2 + x
    q = z + z * x
    h = (z * z)
    y = h + q + q * x
    y.backward()
    xpt, ypt = x, y


    # forward pass went well
    assert ymg.data == ypt.data.item()
    # backward pass went well
    print(xmg, xpt, xpt.grad)
    assert xmg.grad == xpt.grad.item()


def test_more_ops():

    a = Value(-4.0)
    b = Value(2.0)
    c = a + b
    d = a * b + b**3
    c += c + 1
    c += 1 + c + (-a)
    d += d * 2 + (b + a)
    d += 3 * d + (b - a)
    e = c - d
    f = e**2
    g = f / 2.0
    g += 10.0 / f
    g.backward()
    amg, bmg, gmg = a, b, g

    a = torch.Tensor([-4.0]).double()
    b = torch.Tensor([2.0]).double()
    a.requires_grad = True
    b.requires_grad = True
    c = a + b
    d = a * b + b**3
    c = c + c + 1
    c = c + 1 + c + (-a)
    d = d + d * 2 + (b + a)
    d = d + 3 * d + (b - a)
    e = c - d
    f = e**2
    g = f / 2.0
    g = g + 10.0 / f
    g.backward()
    apt, bpt, gpt = a, b, g

    tol = 1e-6
    # forward pass went well
    assert abs(gmg.data - gpt.data.item()) < tol
    # backward pass went well
    assert abs(amg.grad - apt.grad.item()) < tol
    assert abs(bmg.grad - bpt.grad.item()) < tol

In [297]:
test_sanity_check()

Value(data=-4.0, grad=27.0) tensor([-4.], dtype=torch.float64, requires_grad=True) tensor([27.], dtype=torch.float64)


In [298]:
test_more_ops()